In [24]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import os
from copy import copy
from data_and_events import data_and_events

In [ ]:
class main(data_and_events):
    def __init__(self, image_name) -> None:
        super().__init__(image_name)

    def run(image_name: str):
        main_obj = data_and_events(image_name)
        cv.namedWindow("image")
        cv.setMouseCallback("image", main_obj.mouse_callback)
        while True:
            img = main_obj.get_image()
            cv.imshow("image", img)
            if main_obj.keyboard_handler() == -1:
                break
        cv.destroyAllWindows()

In [ ]:
main_obj = main("lantern.jpg")
main.run("lantern.jpg")

rectangle drawn
top left:  346
top right:  51
bottom left:  908
bottom right:  790
